In [ ]:
def group_and_average(cleaned_dataframe):
    import pandas as pd
    import warnings
    warnings.filterwarnings("ignore", category=FutureWarning, module="pandas")

    # Extract the 'phon_R01_Sxx' prefix from the 'name' column
    cleaned_dataframe['subject_prefix'] = cleaned_dataframe['name'].str.extract(r'(phon_R01_S\d+)')

    # Define a function to remove outliers using IQR
    def remove_outliers(group):
        Q1 = group.quantile(0.25)
        Q3 = group.quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        return group[(group >= lower_bound) & (group <= upper_bound)]

    # Remove outliers for each numeric column
    numeric_columns = cleaned_dataframe.select_dtypes(include=['number']).columns
    cleaned_dataframe[numeric_columns] = cleaned_dataframe.groupby('subject_prefix')[numeric_columns].apply(remove_outliers)

    # Define the group_and_average_2 function with the numeric_only parameter
    def group_and_average_2(group):
        # This function takes a group of trials for a subject and returns a row with the averaged variables
        return group.mean(numeric_only=True)

    # Group and average by subject
    aggregated_dataframe = cleaned_dataframe.groupby('subject_prefix').apply(group_and_average_2)

    # Assign the index to the 'name' column
    aggregated_dataframe['name'] = aggregated_dataframe.index

    # Reorganize the columns to have 'name' at the beginning
    aggregated_dataframe = aggregated_dataframe[['name'] + [col for col in aggregated_dataframe.columns if col != 'name']]

    # Reset the index to have a flat DataFrame
    aggregated_dataframe.reset_index(drop=True, inplace=True)

    # Drop the auxiliary column 'subject_prefix' from the original DataFrame
    cleaned_dataframe.drop('subject_prefix', axis=1, inplace=True)

    return aggregated_dataframe